# スキャン画像の読み取り，位置合わせ

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

%config InlineBackend.figure_formats = {'png', 'retina'}

### 画像の読み込み

In [0]:
import glob
import os

def read_images(dirname, ext=None):
    if ext:
        pathr = os.path.join(dirname, "**", "*"+ext)
    else:
        pathr = os.path.join(dirname, "**")
    paths = glob.iglob(pathr, recursive=True)
    for p in paths:
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        yield img

### 変換関数

In [0]:
def find_markers(img, mask):
    masked = img * mask
    ret,thresh = cv2.threshold(masked, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    sort_idx = np.argsort(areas)[::-1]



def align_image(base_markers, img_markers, img):
    w, h = img.shape
    M = cv2.getAffineTransform(img_markers, base_markers)
    dst = cv2.warpAffine(img, M, (h, w), borderValue=255)
    return dst